In [ ]:
#from google.colab import drive,files
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

#drive.mount('/content/drive')

# Klasör yolunu girin
workspace_path = "C:/Users/celal/Documents/Github/goruntu-isleme-projesi/"

# Klasörü workspace olarak ayarlayın
os.chdir(workspace_path)

In [ ]:
# Kaggle API anahtarını Colab'a yükleme
uploaded = files.upload()

# API anahtarını yerel dizine taşıma
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Veri setlerini indirme
!kaggle datasets download -d undersc0re/fake-vs-real-face-classification

# Zip dosyalarının taşınacağı dizinleri tanımla
fake_vs_real_zip = workspace_path + 'fake-vs-real-face-classification.zip'

# Çıkarılacak dizinleri tanımla
fake_vs_real_folder = workspace_path + 'real-vs-fake'

# Gerekli klasörleri oluştur
os.makedirs(fake_vs_real_folder, exist_ok=True)

# Zip dosyalarını çıkar
!unzip -q $fake_vs_real_zip -d $fake_vs_real_folder


In [ ]:
train_dir = workspace_path+ 'real-vs-fake/train/'
valid_dir = workspace_path+'real-vs-fake/Validation/'
test_dir = workspace_path+'real-vs-fake/test/'

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [ ]:
train_transforms = transforms.Compose([transforms.Resize((512,512)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomAutocontrast(0.3),
                                       transforms.RandomAffine(5),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize((512,512)),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize((512,512)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=4)

In [ ]:
import os
import matplotlib.pyplot as plt

class_names = os.listdir(train_dir)
train_image_count = {}
valid_image_count = {}
test_image_count = {}

for i in class_names:
    train_image_count[i] = len(os.listdir(os.path.join(train_dir,i)))

for i in class_names:
    valid_image_count[i] = len(os.listdir(os.path.join(valid_dir,i)))

print(f"TRAIN: {train_image_count}")
print(f"VALIDATION: {valid_image_count}")

In [ ]:
import matplotlib

mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

def imshow(inp, size =(40,40), title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = mean_nums
    std = std_nums
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=size)
    plt.imshow(inp)
    if title is not None:
        plt.title(title, size=30)
    plt.pause(0.001)  # pause a bit so that plots are updated


def visualize_model(model, num_images=9):
    was_training = model.training
    model.eval()
    images_handeled = 0
    ax = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(testloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_handeled += 1
                ax = plt.subplot(num_images, 2, images_handeled)
                ax.axis('off')
                ax.set_title('Actual: {} Predicted: {}'.format(class_names[labels[j].item()],class_names[preds[j]]))
                imshow(inputs.cpu().data[j], (16,4))

                if images_handeled == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
# visualize_model(model,len(testloader))
# plt.show()

In [ ]:
sonucKlasor = workspace_path + 'sonuclar/'


In [ ]:
mobileNetV3Sonuc = sonucKlasor + "MobileNetV3/"

In [ ]:
mobileNetV3Sonuc

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Önceden eğitilmiş modeli yükleyin
MobileNetV3 = models.mobilenet_v3_large(pretrained="MobileNet_V3_Large_Weights.IMAGENET1K_V1")
num_ftrs = MobileNetV3.classifier[3].in_features
MobileNetV3.classifier[3].in_features = nn.Linear(num_ftrs, 2)
model = MobileNetV3
# Tüm katmanları dondurun
# for param in model.parameters():
#     param.requires_grad = False

# Cihaz tipini tanımlayın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli cihaza taşıyın
model.to(device)

# Kayıp fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Eğitim döngüsü
epochs = 20
best_accuracy = 0.0
for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0

    # Eğitim aşaması
    model.train()
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    # Doğrulama aşaması
    model.eval()
    for data, target in validloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)

        # Doğruluğu hesaplayın
        prob = torch.exp(output)
        top_p, top_class = prob.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    # Sonuçları yazdırın
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)
    accuracy = accuracy/len(validloader)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.2f}%'.format(epoch+1, train_loss, valid_loss, accuracy*100))

    torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/MobileNetV3/{epoch+1}epoch.pth")
    if accuracy >= best_accuracy:
        print("Saving model with validation accuracy: {:.2f}%".format(accuracy*100))
        best_accuracy = accuracy
        torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/MobileNetV3/best_model.pth")

In [10]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Önceden eğitilmiş modeli yükleyin
MobileNetV3small = models.mobilenet_v3_small(pretrained="MobileNet_V3_Small_Weights.IMAGENET1K_V1")
num_ftrs = MobileNetV3small.classifier[3].in_features
MobileNetV3small.classifier[3].in_features = nn.Linear(num_ftrs, 2)
model = MobileNetV3small
# Tüm katmanları dondurun
# for param in model.parameters():
#     param.requires_grad = False

# Cihaz tipini tanımlayın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli cihaza taşıyın
model.to(device)

# Kayıp fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Eğitim döngüsü
epochs = 20
best_accuracy = 0.0
for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0

    # Eğitim aşaması
    model.train()
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    # Doğrulama aşaması
    model.eval()
    for data, target in validloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)

        # Doğruluğu hesaplayın
        prob = torch.exp(output)
        top_p, top_class = prob.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    # Sonuçları yazdırın
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)
    accuracy = accuracy/len(validloader)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.2f}%'.format(epoch+1, train_loss, valid_loss, accuracy*100))

    torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/MobileNetV3small/{epoch+1}epoch.pth")
    if accuracy >= best_accuracy:
        print("Saving model with validation accuracy: {:.2f}%".format(accuracy*100))
        best_accuracy = accuracy
        torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/MobileNetV3small/best_model.pth")

Epoch: 2 	Training Loss: 0.677820 	Validation Loss: 0.908078 	Validation Accuracy: 56.25%
Epoch: 3 	Training Loss: 0.615068 	Validation Loss: 0.815012 	Validation Accuracy: 59.18%
Epoch: 4 	Training Loss: 0.578109 	Validation Loss: 0.668739 	Validation Accuracy: 66.60%
Saving model with validation accuracy: 66.60%
Epoch: 5 	Training Loss: 0.542675 	Validation Loss: 0.691266 	Validation Accuracy: 64.45%
Epoch: 6 	Training Loss: 0.514837 	Validation Loss: 0.758758 	Validation Accuracy: 65.62%
Epoch: 7 	Training Loss: 0.492084 	Validation Loss: 0.676669 	Validation Accuracy: 64.06%
Epoch: 8 	Training Loss: 0.451071 	Validation Loss: 0.734401 	Validation Accuracy: 67.19%
Saving model with validation accuracy: 67.19%
Epoch: 9 	Training Loss: 0.425306 	Validation Loss: 1.293247 	Validation Accuracy: 58.59%
Epoch: 10 	Training Loss: 0.407713 	Validation Loss: 0.730837 	Validation Accuracy: 67.38%
Saving model with validation accuracy: 67.38%
Epoch: 11 	Training Loss: 0.362108 	Validation Loss

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Önceden eğitilmiş modeli yükleyin
SqueezeNet = models.squeezenet1_1(pretrained=True)
num_classes = 2  # Sınıf sayısına uygun olarak ayarlayın

# SqueezeNet'in son tam bağlı katmanını değiştirin
SqueezeNet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
SqueezeNet.num_classes = num_classes

model = SqueezeNet

# Tüm katmanları dondurun
# for param in model.parameters():
#     param.requires_grad = False

# Cihaz tipini tanımlayın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli cihaza taşıyın
model.to(device)

# Kayıp fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Eğitim döngüsü
epochs = 20
best_accuracy = 0.0
for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0

    # Eğitim aşaması
    model.train()
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    # Doğrulama aşaması
    model.eval()
    for data, target in validloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)

        # Doğruluğu hesaplayın
        prob = torch.exp(output)
        top_p, top_class = prob.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    # Sonuçları yazdırın
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)
    accuracy = accuracy/len(validloader)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.2f}%'.format(epoch+1, train_loss, valid_loss, accuracy*100))

    torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/SqueezeNet/{epoch+1}epoch.pth")
    if accuracy >= best_accuracy:
        print("Saving model with validation accuracy: {:.2f}%".format(accuracy*100))
        best_accuracy = accuracy
        torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/SqueezeNet/best_model.pth")


c:\Users\celal\.conda\envs\torchgpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1 	Training Loss: 0.694039 	Validation Loss: 0.684581 	Validation Accuracy: 56.05%
Saving model with validation accuracy: 56.05%
Epoch: 2 	Training Loss: 0.692219 	Validation Loss: 0.688950 	Validation Accuracy: 55.47%
Epoch: 3 	Training Loss: 0.686017 	Validation Loss: 0.684681 	Validation Accuracy: 54.30%
Epoch: 4 	Training Loss: 0.681390 	Validation Loss: 0.685954 	Validation Accuracy: 55.27%
Epoch: 5 	Training Loss: 0.680362 	Validation Loss: 0.695654 	Validation Accuracy: 49.61%
Epoch: 6 	Training Loss: 0.678296 	Validation Loss: 0.692454 	Validation Accuracy: 53.32%
Epoch: 7 	Training Loss: 0.681009 	Validation Loss: 0.681822 	Validation Accuracy: 55.66%
Epoch: 8 	Training Loss: 0.672251 	Validation Loss: 0.710692 	Validation Accuracy: 50.20%
Epoch: 9 	Training Loss: 0.667542 	Validation Loss: 0.680944 	Validation Accuracy: 56.25%
Saving model with validation accuracy: 56.25%
Epoch: 10 	Training Loss: 0.658907 	Validation Loss: 0.671644 	Validation Accuracy: 56.05%
Epoch: 

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Önceden eğitilmiş modeli yükleyin
ShuffleNet = models.shufflenet_v2_x1_0(pretrained=True)
num_classes = 2  # Sınıf sayısına uygun olarak ayarlayın

# ShuffleNet'in son tam bağlı katmanını değiştirin
ShuffleNet.fc = nn.Linear(1024, num_classes)

model = ShuffleNet

# Tüm katmanları dondurun
# for param in model.parameters():
#     param.requires_grad = False

# Cihaz tipini tanımlayın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli cihaza taşıyın
model.to(device)

# Kayıp fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Eğitim döngüsü
epochs = 20
best_accuracy = 0.0
for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0

    # Eğitim aşaması
    model.train()
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    # Doğrulama aşaması
    model.eval()
    for data, target in validloader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)

        # Doğruluğu hesaplayın
        prob = torch.exp(output)
        top_p, top_class = prob.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    # Sonuçları yazdırın
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)
    accuracy = accuracy/len(validloader)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.2f}%'.format(epoch+1, train_loss, valid_loss, accuracy*100))

    torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/ShuffleNet/{epoch+1}epoch.pth")
    if accuracy >= best_accuracy:
        print("Saving model with validation accuracy: {:.2f}%".format(accuracy*100))
        best_accuracy = accuracy
        torch.save(model.state_dict(), f"C:/Users/celal/Documents/Github/goruntu-isleme-projesi/sonuclar/ShuffleNet/best_model.pth")


c:\Users\celal\.conda\envs\torchgpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to C:\Users\celal/.cache\torch\hub\checkpoints\shufflenetv2_x1-5666bf0f80.pth
100%|██████████| 8.79M/8.79M [00:01<00:00, 5.09MB/s]


Epoch: 1 	Training Loss: 0.688730 	Validation Loss: 0.686953 	Validation Accuracy: 55.47%
Saving model with validation accuracy: 55.47%
Epoch: 2 	Training Loss: 0.687278 	Validation Loss: 0.686996 	Validation Accuracy: 55.47%
Saving model with validation accuracy: 55.47%
Epoch: 3 	Training Loss: 0.687020 	Validation Loss: 0.686649 	Validation Accuracy: 55.47%
Saving model with validation accuracy: 55.47%
Epoch: 4 	Training Loss: 0.687134 	Validation Loss: 0.686375 	Validation Accuracy: 55.47%
Saving model with validation accuracy: 55.47%
Epoch: 5 	Training Loss: 0.686577 	Validation Loss: 0.686128 	Validation Accuracy: 55.47%
Saving model with validation accuracy: 55.47%
Epoch: 6 	Training Loss: 0.686408 	Validation Loss: 0.686128 	Validation Accuracy: 55.47%
Saving model with validation accuracy: 55.47%
Epoch: 7 	Training Loss: 0.686384 	Validation Loss: 0.685710 	Validation Accuracy: 55.47%
Saving model with validation accuracy: 55.47%
Epoch: 8 	Training Loss: 0.685294 	Validation Lo